In [ ]:
import requests
import re
from collections import defaultdict
import json

requirements_re = re.compile(r'^([\w.-]+)\s*([<>=!~]+)\s*([\w.-]+)$')

# Test cases
test_cases = [
    'requests==2.22.0',
    'numpy>=1.18.1',
    'django<3.0',
    'python-dateutil>=2.8.1'
]

for req in test_cases:
    match = requirements_re.match(req)
    if match:
        name, comp, version = match.groups()
        print(f"Package: {name}, Comparator: {comp}, Version: {version}")

In [ ]:
# Python requirements NAME [COMPARATOR VERSION]

# requirements_re = re.compile(r'^([\w_-]+)([<>=!]+)([\w]+)$') << wrong
# requirements_re.match('requests==2.22.0')

equirements_re = re.compile(r'^([\w_-]+)([<>=!]+)([\w.]+)$')

In [ ]:
with open('./checkme/constraints.txt', 'r') as fd: 
    lines = fd.read().splitlines()

constraints = {}
for line in lines:
    match = requirements_re.match(line)
    
    if not match:
        print(f"Invalid requirement: {line}")
        continue
        
    name, comp, version = match.groups()
    constraints[name] = (comp, version)
constraints


In [ ]:
with open('./repos-req.txt', 'r') as fd: 
    lines = fd.read().splitlines()

requirements = {}
for line in lines:
    match = requirements_re.match(line)
    
    if not match:
        print(f"Invalid requirement: {line}")
        continue
        
    name, comp, version = match.groups()
    requirements[name] = (comp, version)
requirements


In [ ]:
# requirements
scores  = {
    pkg: requests.get(f'http://localhost:3000/api/package/pypi/{pkg}').json() for pkg in constraints.keys()
}

In [ ]:
scores['deprecation']

In [ ]:
sd = lambda: defaultdict(lambda: set())

In [ ]:
maturity_scores = {"direct": sd(), "indirect": sd()}
health_risk_scores = {"direct": sd(), "indirect": sd()}
maturity_notes = defaultdict(lambda: set())
health_risk_notes = defaultdict(lambda: set())
for pkg_name, score in scores.items():
    where = 'direct' if pkg_name in requirements else 'indirect'
    if 'error' in score:
        maturity_scores[where]['Unknown'].add(pkg_name)
        continue
    maturity_scores[where][score['source']['maturity']['value']].add(pkg_name)
    health_risk_scores[where][score['source']['health_risk']['value']].add(pkg_name)

    for note in score['source']['health_risk']['notes']:
        health_risk_notes[note].add(pkg_name)

In [ ]:

print(f'maturity   ', " ".join([f'{k}: {len(v)}' for k,v in maturity_scores.items()]))
print(f'health risk', " ".join([f'{k}: {len(v)}' for k,v in health_risk_scores.items()]))

In [ ]:
# .update(maturity_scores.pop(['Not Found']))
nf = 


In [ ]:
maturity_scores['Unknown'].update(maturity_scores.pop('Not Found'))

In [ ]:
def json_default(obj):
    if isinstance(obj, set):
        return list(obj)
    return obj

print(json.dumps({
    "health_risk": {"direct": {k:len(v) for k,v in health_risk_scores['direct'].items()},
                    "indirect": {k:len(v) for k,v in health_risk_scores['indirect'].items()},
                    },
    "maturity_scores": {
        "direct": {k:len(v) for k,v in maturity_scores['direct'].items()},
        "indirect": {k:len(v) for k,v in maturity_scores['indirect'].items()},
                        },
}, indent=2, default=json_default))

In [ ]:
{k:list(v) for k,v in health_risk_notes.items()}

In [ ]:
print(json.dumps({k:list(v) for k,v in health_risk_notes.items()}, indent=2))

In [ ]:
for note, pkgs in health_risk_notes.items():
    print(note, len(pkgs))
    for pkg in pkgs:
        print(f'  {pkg}') 